In [1]:
from keras.optimizers import Adam
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from tensorflow import keras
import os
import tempfile
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from tensorflow.keras.regularizers import l1, l2, L1L2



Using TensorFlow backend.


In [ ]:
# smote and upsampling + validation set pulled from training set (now we have data leakage)
filepath_files = '/Users/krahman/work/tutorials/tensorflow_classification/data/'

train_samples = pd.read_csv(filepath_files + 'mod_x_train.csv').drop('Unnamed: 0', axis=1)
train_labels = pd.read_csv(filepath_files + 'mod_y_train.csv').drop('Unnamed: 0', axis=1)
train_samples = pd.concat([train_samples, train_labels], axis=1)

test_samples = pd.read_csv(filepath_files + 'mod_x_test.csv').drop('Unnamed: 0', axis=1)
test_labels = pd.read_csv(filepath_files + 'mod_y_test.csv').drop('index', axis=1)
test_samples = pd.concat([test_samples, test_labels], axis=1)

test_samples = shuffle(test_samples).reset_index(drop=True)

neg, pos = np.bincount(train_samples['0'])
initial_bias = np.log([pos/neg])
output_bias = tf.keras.initializers.Constant(initial_bias)

test_samples, val_samples = train_test_split(test_samples, test_size=.2)

train_labels = np.array(train_samples.pop('0'))
test_labels = np.array(test_samples.pop('0'))
val_labels = np.array(val_samples.pop('0'))

print("Training data shape:", train_samples.shape)
print("Validation data shape:", val_samples.shape)
print("Testing data shape:", test_samples.shape)

train_samples = np.array(train_samples)
test_samples = np.array(test_samples)
val_samples = np.array(val_samples)

# scaler = MinMaxScaler(feature_range=(0,1))
scaler = StandardScaler()
scaled_train_samples = scaler.fit_transform(train_samples)
scaled_test_samples = scaler.transform(test_samples)
scaled_val_samples = scaler.transform(val_samples)

loss = keras.losses.BinaryCrossentropy()

In [ ]:
# # smote and upsampling + validation set pulled from training set (now we have data leakage)
# from sklearn.utils import shuffle
# import numpy as np
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler

# filepath_data = '/Users/krahman/work/tutorials/tensorflow_classification/data/'

# train_sample = pd.read_csv(filepath_data + 'mod_x_train.csv').drop('Unnamed: 0', axis=1)
# train_label = pd.read_csv(filepath_data + 'mod_y_train.csv').drop('Unnamed: 0', axis=1)
# train_sample = pd.concat([train_sample, train_label], axis=1)

# test_sample = pd.read_csv(filepath_data + 'mod_x_test.csv').drop('Unnamed: 0', axis=1)
# test_label = pd.read_csv(filepath_data + 'mod_y_test.csv').drop('index', axis=1)
# test_sample = pd.concat([test_sample, test_label], axis=1)

# list_data = [train_sample, test_sample]



# class PreProcessing():
#     def __init__(self, list_data):
# #         self._parse_list_data(list_data)
#         self.process_data(list_data)
#         self.train_sample = list_data[0]
#         self.test_sample = list_data[1]

#         # NEXT, call the self.val_sample here have it return from split. 
# #     def _parse_list_data(self, list_data):
# #         self.train_sample = list_data[0]
# #         self.test_sample = list_data[1]
        
#     def process_data(self, list_data):
#         self.train_sample = list_data[0]
#         self.test_sample = list_data[1]
#         self._shuffle()
#         self._split_test_data()
#         self._create_target()
#         self._training_sets_array()
#         self._scale_data()
        
#     def _shuffle(self):
#         self.test_sample = shuffle(self.test_sample).reset_index(drop=True)

    
#     def _split_test_data(self):
#         self.test_sample, self.val_sample = train_test_split(self.test_sample,
#                                                              test_size=.2)
        
#     def _create_target(self):
#         self.train_label = np.array(self.train_sample.pop('0'))
#         self.test_label = np.array(self.test_sample.pop('0'))
#         self.val_label = np.array(self.val_sample.pop('0'))
#         self._print_summary()
        
#     def _print_summary(self):
#         print("Training data shape:", self.train_sample.shape)
#         print("Validation data shape:", self.val_sample.shape)
#         print("Testing data shape:", self.test_sample.shape)
        
#     def _training_sets_array(self):
#         self.train_sample = np.array(self.train_sample)
#         self.test_sample = np.array(self.test_sample)
#         self.val_sample = np.array(self.val_sample)
        
#     def _scale_data(self):
#         scaler = StandardScaler()
#         self.scaled_train_sample = scaler.fit_transform(self.train_sample)
#         self.scaled_test_sample = scaler.transform(self.test_sample)
#         self.scaled_val_sample = scaler.transform(self.val_sample)

# pp = PreProcessing(list_data)


In [2]:
# smote and upsampling + validation set pulled from training set (now we have data leakage)
from sklearn.utils import shuffle
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

filepath_data = '/Users/krahman/work/tutorials/tensorflow_classification/data/'

train_sample = pd.read_csv(filepath_data + 'mod_x_train.csv').drop('Unnamed: 0', axis=1)
train_label = pd.read_csv(filepath_data + 'mod_y_train.csv').drop('Unnamed: 0', axis=1)
train_sample = pd.concat([train_sample, train_label], axis=1)
test_sample = pd.read_csv(filepath_data + 'mod_x_test.csv').drop('Unnamed: 0', axis=1)
test_label = pd.read_csv(filepath_data + 'mod_y_test.csv').drop('index', axis=1)
test_sample = pd.concat([test_sample, test_label], axis=1)

target = '0'

list_data = [train_sample, test_sample]

class PreProcessing():
    def __init__(self, list_data, target):
        self.target = target
        self.train_sample = list_data[0]
        self.test_sample = list_data[1]
        self.test_sample = self.shuffle_data(self.test_sample)
        self.test_sample, self.val_sample = self._split_test_data()
        self.train_label = self._create_target(self.train_sample)
        self.test_label = self._create_target(self.test_sample)
        self.val_label = self._create_target(self.val_sample)
        self.process_data()
        
    def process_data(self):
        self._print_summary()
        self._training_sets_array()
        self._scale_data()
        
    def shuffle_data(self, dataset):
        return shuffle(dataset).reset_index(drop=True)
    
    def _split_test_data(self):
        return train_test_split(self.test_sample, test_size=.2)
        
    def _create_target(self, dataset):
        return np.array(dataset.pop(self.target))
        
    def _print_summary(self):
        print("Training data shape:", self.train_sample.shape)
        print("Testing data shape:", self.test_sample.shape)
        print("Validation data shape:", self.val_sample.shape)
        print("train_label length:", self.train_label.shape[0])
        print("test_label length:", self.test_label.shape[0])
        print("val_label length:", self.val_label.shape[0])
        
    def _training_sets_array(self):
        self.train_sample = np.array(self.train_sample)
        self.test_sample = np.array(self.test_sample)
        self.val_sample = np.array(self.val_sample)
        
    def _scale_data(self):
        scaler = StandardScaler()
        self.train_sample = scaler.fit_transform(self.train_sample)
        self.test_sample = scaler.transform(self.test_sample)
        self.val_sample = scaler.transform(self.val_sample)

pp = PreProcessing(list_data, target)


Training data shape: (190000, 233)
Testing data shape: (47243, 233)
Validation data shape: (11811, 233)
train_label length: 190000
test_label length: 47243
val_label length: 11811


In [4]:
model.predict_classes(pp.test_sample, batch_size=10, verbose=0)

array([[0],
       [0],
       [1],
       ...,
       [0],
       [0],
       [0]], dtype=int32)

In [6]:
model.predict(pp.test_sample, batch_size=10, verbose=0)

array([[1.04543716e-01],
       [1.68702602e-02],
       [9.99288142e-01],
       ...,
       [2.81370711e-03],
       [5.99788063e-06],
       [1.62345372e-04]], dtype=float32)

In [3]:
# Final model WORKING
list_auc_score = []
path_savedmodel = '/Users/krahman/work/tutorials/tensorflow_classification/saved_models_2/'
model = tf.keras.models.load_model(path_savedmodel)

# predictions = model.predict(pp.test_sample, batch_size=10, verbose=0)
rounded_predictions = model.predict_classes(pp.test_sample, batch_size=10, verbose=0)
fpr, tpr, thresholds = roc_curve(pp.test_label, rounded_predictions, pos_label=1)
auc_score = auc(fpr, tpr)
list_auc_score.append(auc_score)
print('auc_score:', auc_score)    
cm = confusion_matrix(pp.test_label, rounded_predictions)
print(cm)

auc_score: 0.8592748711112961
[[42225  3337]
 [  350  1331]]


In [ ]:
import tensorflow as tf
import numpy as np 

neg, pos = np.bincount(train_samples['0'])
initial_bias = np.log([pos/neg])
output_bias = tf.keras.initializers.Constant(initial_bias)

loss = keras.losses.BinaryCrossentropy()

In [ ]:
# NEXT, should we load model weights and use this as our method? 
# should we just load the model? Well, we do want to show them all our details, so we should do this all.
checkpoint_path = "./cp.ckpt.testing2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, 
                                                 monitor='val_auc', 
                                                 verbose=1,
                                                 save_best_only=False, 
                                                 save_weights_only=True,
                                                 mode='max',
                                                 save_freq='epoch')


# creating model
epochs = 30
lr = .0001
metrics = [keras.metrics.AUC(name='auc'),
           keras.metrics.FalsePositives(name='fp'),
           keras.metrics.FalseNegatives(name='fn')]

model = keras.Sequential([keras.layers.Dense(256, activation='relu', kernel_regularizer=l2(0.0001), input_shape=(train_samples.shape[-1],)),
                          keras.layers.Dense(256, activation='relu', kernel_regularizer=l2(0.0001)),
                          keras.layers.Dense(256, activation='relu', kernel_regularizer=l2(0.0001)),
                          keras.layers.Dense(256, activation='relu', kernel_regularizer=l2(0.0001)),
                          keras.layers.Dense(256, activation='relu', kernel_regularizer=l2(0.0001)),
                          keras.layers.Dense(256, activation='relu', kernel_regularizer=l2(0.0001)),
                          keras.layers.Dense(256, activation='relu', kernel_regularizer=l2(0.0001)),
                          keras.layers.Dropout(.5),
                          keras.layers.Dense(1, activation='sigmoid', bias_initializer=output_bias)])

model.compile(optimizer=keras.optimizers.Adam(lr=lr), loss=loss, metrics=metrics)


# saving weights for checkpoints
model.save_weights(checkpoint_path.format(epoch=0))


# fitting model
model.fit(scaled_train_samples, train_labels, validation_data=(scaled_val_samples, val_labels), 
          batch_size=20, epochs=epochs, shuffle=True, verbose=2, workers=16, 
          use_multiprocessing=True,
          callbacks=[cp_callback])


# now that all epochs have completed, we will test load and test each weight. 
# parsing through folder with saved weights and selecting unique values with cp-0001 to cp-xxxx to create
# a list of all model weights to parse through for testing results against the test set. 
checkpoint_path = './cp.ckpt.testing2/'
list_checkpoints = []
for file in os.listdir(checkpoint_path):
    val_checkpoint = file[0:12]
    if file[0:3]=='cp-' and val_checkpoint not in list_checkpoints:
        list_checkpoints.append(val_checkpoint)
list_checkpoints.sort()


# parsing through each saved weights through the list, then loading the weights, scoring and plotting results.
filepath_checkpoint_folder = './cp.ckpt.testing2/'
list_auc_score = []
list_epoch = []
val_epoch = 0

for chkpt in list_checkpoints:
    filepath_chkpt = filepath_checkpoint_folder + chkpt

    model = keras.Sequential([keras.layers.Dense(256, activation='relu', kernel_regularizer=l2(0.0001), input_shape=(train_samples.shape[-1],)),
                              keras.layers.Dense(256, activation='relu', kernel_regularizer=l2(0.0001)),
                              keras.layers.Dense(256, activation='relu', kernel_regularizer=l2(0.0001)),
                              keras.layers.Dense(256, activation='relu', kernel_regularizer=l2(0.0001)),
                              keras.layers.Dense(256, activation='relu', kernel_regularizer=l2(0.0001)),
                              keras.layers.Dense(256, activation='relu', kernel_regularizer=l2(0.0001)),
                              keras.layers.Dense(256, activation='relu', kernel_regularizer=l2(0.0001)),
                              keras.layers.Dropout(.5),
                              keras.layers.Dense(1, activation='sigmoid', bias_initializer=output_bias)])

    
    model.load_weights(filepath_chkpt)
    predictions = model.predict(scaled_test_samples, batch_size=10, verbose=0)
    rounded_predictions = model.predict_classes(scaled_test_samples, batch_size=10, verbose=0)
    fpr, tpr, thresholds = roc_curve(test_labels, rounded_predictions, pos_label=1)
    auc_score = auc(fpr, tpr)
    list_auc_score.append(auc_score)
    val_epoch = val_epoch + 1
    print('epoch:', val_epoch, 'auc_score:', auc_score)    
    cm = confusion_matrix(test_labels, rounded_predictions)
    
list_epoch = []
for val in range(1,len(list_auc_score) + 1):
    list_epoch.append(val)
    
plt.plot(list_epoch, list_auc_score)

In [ ]:
# Final model - for non OOP dataset
list_auc_score = []
path_savedmodel = '/Users/krahman/work/tutorials/tensorflow_classification/saved_models_2/'
model = tf.keras.models.load_model(path_savedmodel)

predictions = model.predict(scaled_test_samples, batch_size=10, verbose=0)
rounded_predictions = model.predict_classes(scaled_test_samples, batch_size=10, verbose=0)
fpr, tpr, thresholds = roc_curve(test_labels, rounded_predictions, pos_label=1)
auc_score = auc(fpr, tpr)
list_auc_score.append(auc_score)
# val_epoch = val_epoch + 1
print('auc_score:', auc_score)    
cm = confusion_matrix(test_labels, rounded_predictions)

In [ ]:
# epochs = 9
for neurons in [256]:
    print('4 layer, neurons:', neurons)
    metrics = [keras.metrics.AUC(name='auc'),
               keras.metrics.FalsePositives(name='fp'),
               keras.metrics.FalseNegatives(name='fn')]

    epochs = 9
    lr = .001

    model = keras.Sequential([keras.layers.Dense(neurons, activation='relu', input_shape=(train_samples.shape[-1],)),
                              keras.layers.Dense(neurons, activation='relu'),
                              keras.layers.Dense(neurons, activation='relu'),
                              keras.layers.Dense(neurons, activation='relu'),
                              keras.layers.Dense(neurons, activation='relu'),
                              keras.layers.Dense(1, activation='sigmoid', bias_initializer=output_bias)])

    model.compile(optimizer=keras.optimizers.Adam(lr=lr), loss=loss, metrics=metrics)
    model.fit(scaled_train_samples, train_labels, validation_data=(scaled_val_samples, val_labels), 
              batch_size=20, epochs=epochs, shuffle=True, verbose=2, workers=16, 
              use_multiprocessing=True)

    predictions = model.predict(scaled_test_samples, batch_size=10, verbose=0)
    rounded_predictions = model.predict_classes(scaled_test_samples, batch_size=10, verbose=0)
    fpr, tpr, thresholds = roc_curve(test_labels, rounded_predictions, pos_label=1)
    auc_score = auc(fpr, tpr)
    print('auc score:\n', auc_score)
    cm = confusion_matrix(test_labels, rounded_predictions)
    print(cm)

In [ ]:
# os.
# we need to graph the training, validation and test results  

In [ ]:
# TESTING
# creating Modelcheckpoint
checkpoint_path = "./cp.ckpt.testing2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, 
                                                 monitor='val_auc', 
                                                 verbose=1,
                                                 save_best_only=False, 
                                                 save_weights_only=True,
                                                 mode='max',
                                                 save_freq='epoch')


# creating model
epochs = 10
lr = .0001
metrics = [keras.metrics.AUC(name='auc'),
           keras.metrics.FalsePositives(name='fp'),
           keras.metrics.FalseNegatives(name='fn')]

model = keras.Sequential([keras.layers.Dense(16, activation='relu', kernel_regularizer=l2(0.0001), input_shape=(train_samples.shape[-1],)),
                          keras.layers.Dense(16, activation='relu', kernel_regularizer=l2(0.0001),bias_regularizer=l2(0.0001)),
                          keras.layers.Dropout(.5),
                          keras.layers.Dense(1, activation='sigmoid', bias_initializer=output_bias)])

# Dense(units, activation=None, use_bias=True, kernel_initializer='glorot_uniform',
#       bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None,
#       activity_regularizer=None, kernel_constraint=None, bias_constraint=None
    
model.compile(optimizer=keras.optimizers.Adam(lr=lr), loss=loss, metrics=metrics)


# saving weights for checkpoints
model.save_weights(checkpoint_path.format(epoch=0))


# fitting model
model.fit(scaled_train_samples, train_labels, validation_data=(scaled_val_samples, val_labels), 
          batch_size=20, epochs=epochs, shuffle=True, verbose=2, workers=16, 
          use_multiprocessing=True,
          callbacks=[cp_callback])


# now that all epochs have completed, we will test load and test each weight. 
# parsing through folder with saved weights and selecting unique values with cp-0001 to cp-xxxx to create
# a list of all model weights to parse through for testing results against the test set. 
checkpoint_path = './cp.ckpt.testing2/'
list_checkpoints = []
for file in os.listdir(checkpoint_path):
    val_checkpoint = file[0:12]
    if file[0:3]=='cp-' and val_checkpoint not in list_checkpoints:
        list_checkpoints.append(val_checkpoint)
list_checkpoints.sort()


# parsing through each saved weights through the list, then loading the weights, scoring and plotting results.
filepath_checkpoint_folder = './cp.ckpt.testing2/'
list_auc_score = []
list_epoch = []
val_epoch = 0

for chkpt in list_checkpoints:
    filepath_chkpt = filepath_checkpoint_folder + chkpt

    model = keras.Sequential([keras.layers.Dense(16, activation='relu', kernel_regularizer=l2(0.0001), input_shape=(train_samples.shape[-1],)),
                              keras.layers.Dense(16, activation='relu', kernel_regularizer=l2(0.0001)),
                              keras.layers.Dropout(.5),
                              keras.layers.Dense(1, activation='sigmoid', bias_initializer=output_bias)])

    
    model.load_weights(filepath_chkpt)
    predictions = model.predict(scaled_test_samples, batch_size=10, verbose=0)
    rounded_predictions = model.predict_classes(scaled_test_samples, batch_size=10, verbose=0)
    fpr, tpr, thresholds = roc_curve(test_labels, rounded_predictions, pos_label=1)
    auc_score = auc(fpr, tpr)
    list_auc_score.append(auc_score)
    val_epoch = val_epoch + 1
    print('epoch:', val_epoch, 'auc_score:', auc_score)    
    cm = confusion_matrix(test_labels, rounded_predictions)
    
list_epoch = []
for val in range(1,len(list_auc_score) + 1):
    list_epoch.append(val)
    
plt.plot(list_epoch, list_auc_score)

In [ ]:
# TESTING
# creating Modelcheckpoint
checkpoint_path = "./cp.ckpt.testing2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, 
                                                 monitor='val_auc', 
                                                 verbose=1,
                                                 save_best_only=False, 
                                                 save_weights_only=True,
                                                 mode='max',
                                                 save_freq='epoch')


# creating model
epochs = 10
lr = .0001
metrics = [keras.metrics.AUC(name='auc'),
           keras.metrics.FalsePositives(name='fp'),
           keras.metrics.FalseNegatives(name='fn')]

model = keras.Sequential([keras.layers.Dense(16, activation='relu', kernel_regularizer=l2(0.0001), input_shape=(train_samples.shape[-1],)),
                          keras.layers.Dense(16, activation='relu', kernel_regularizer=l2(0.0001),
                                             bias_regularizer=l2(0.0001),
#                                              activity_regularizer=l2(0.0001),
#                                              kernel_constraint=l2(0.0001),
#                                              bias_constraint=l2(0.0001)
                                            ),
                          keras.layers.Dropout(.5),
                          keras.layers.Dense(1, activation='sigmoid', bias_initializer=output_bias)])

# Dense(units, activation=None, use_bias=True, kernel_initializer='glorot_uniform',
#       bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None,
#       activity_regularizer=None, kernel_constraint=None, bias_constraint=None
    
model.compile(optimizer=keras.optimizers.Adam(lr=lr), loss=loss, metrics=metrics)


# saving weights for checkpoints
model.save_weights(checkpoint_path.format(epoch=0))


# fitting model
model.fit(scaled_train_samples, train_labels, validation_data=(scaled_val_samples, val_labels), 
          batch_size=20, epochs=epochs, shuffle=True, verbose=2, workers=16, 
          use_multiprocessing=True,
          callbacks=[cp_callback])


# now that all epochs have completed, we will test load and test each weight. 
# parsing through folder with saved weights and selecting unique values with cp-0001 to cp-xxxx to create
# a list of all model weights to parse through for testing results against the test set. 
checkpoint_path = './cp.ckpt.testing2/'
list_checkpoints = []
for file in os.listdir(checkpoint_path):
    val_checkpoint = file[0:12]
    if file[0:3]=='cp-' and val_checkpoint not in list_checkpoints:
        list_checkpoints.append(val_checkpoint)
list_checkpoints.sort()


# parsing through each saved weights through the list, then loading the weights, scoring and plotting results.
filepath_checkpoint_folder = './cp.ckpt.testing2/'
list_auc_score = []
list_epoch = []
val_epoch = 0

for chkpt in list_checkpoints:
    filepath_chkpt = filepath_checkpoint_folder + chkpt

    model = keras.Sequential([keras.layers.Dense(16, activation='relu', kernel_regularizer=l2(0.0001), input_shape=(train_samples.shape[-1],)),
                              keras.layers.Dense(16, activation='relu', kernel_regularizer=l2(0.0001),bias_regularizer=l2(0.0001)),
                              keras.layers.Dropout(.5),
                              keras.layers.Dense(1, activation='sigmoid', bias_initializer=output_bias)])


    
    model.load_weights(filepath_chkpt)
    predictions = model.predict(scaled_test_samples, batch_size=10, verbose=0)
    rounded_predictions = model.predict_classes(scaled_test_samples, batch_size=10, verbose=0)
    fpr, tpr, thresholds = roc_curve(test_labels, rounded_predictions, pos_label=1)
    auc_score = auc(fpr, tpr)
    list_auc_score.append(auc_score)
    val_epoch = val_epoch + 1
    print('epoch:', val_epoch, 'auc_score:', auc_score)    
    cm = confusion_matrix(test_labels, rounded_predictions)
    
list_epoch = []
for val in range(1,len(list_auc_score) + 1):
    list_epoch.append(val)
    
plt.plot(list_epoch, list_auc_score)

In [ ]:
# TESTING
# creating Modelcheckpoint
checkpoint_path = "./cp.ckpt.testing2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, 
                                                 monitor='val_auc', 
                                                 verbose=1,
                                                 save_best_only=False, 
                                                 save_weights_only=True,
                                                 mode='max',
                                                 save_freq='epoch')


# creating model
epochs = 55
lr = .0001
metrics = [keras.metrics.AUC(name='auc'),
           keras.metrics.FalsePositives(name='fp'),
           keras.metrics.FalseNegatives(name='fn')]

model = keras.Sequential([keras.layers.Dense(16, activation='relu', kernel_regularizer=l2(0.0001), input_shape=(train_samples.shape[-1],)),
                          keras.layers.Dense(16, activation='relu', kernel_regularizer=l2(0.0001),
                                             bias_regularizer=l2(0.0001),
                                             activity_regularizer=l2(0.0001),
#                                              kernel_constraint=l2(0.0001),
#                                              bias_constraint=l2(0.0001)
                                            ),
                          keras.layers.Dropout(.5),
                          keras.layers.Dense(1, activation='sigmoid', bias_initializer=output_bias)])

# Dense(units, activation=None, use_bias=True, kernel_initializer='glorot_uniform',
#       bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None,
#       activity_regularizer=None, kernel_constraint=None, bias_constraint=None
    
model.compile(optimizer=keras.optimizers.Adam(lr=lr), loss=loss, metrics=metrics)


# saving weights for checkpoints
model.save_weights(checkpoint_path.format(epoch=0))


# fitting model
model.fit(scaled_train_samples, train_labels, validation_data=(scaled_val_samples, val_labels), 
          batch_size=20, epochs=epochs, shuffle=True, verbose=2, workers=16, 
          use_multiprocessing=True,
          callbacks=[cp_callback])


# now that all epochs have completed, we will test load and test each weight. 
# parsing through folder with saved weights and selecting unique values with cp-0001 to cp-xxxx to create
# a list of all model weights to parse through for testing results against the test set. 
checkpoint_path = './cp.ckpt.testing2/'
list_checkpoints = []
for file in os.listdir(checkpoint_path):
    val_checkpoint = file[0:12]
    if file[0:3]=='cp-' and val_checkpoint not in list_checkpoints:
        list_checkpoints.append(val_checkpoint)
list_checkpoints.sort()


# parsing through each saved weights through the list, then loading the weights, scoring and plotting results.
filepath_checkpoint_folder = './cp.ckpt.testing2/'
list_auc_score = []
list_epoch = []
val_epoch = 0

for chkpt in list_checkpoints:
    filepath_chkpt = filepath_checkpoint_folder + chkpt

    model = keras.Sequential([keras.layers.Dense(16, activation='relu', kernel_regularizer=l2(0.0001), input_shape=(train_samples.shape[-1],)),
                          keras.layers.Dense(16, activation='relu', kernel_regularizer=l2(0.0001),
                                             bias_regularizer=l2(0.0001),
                                             activity_regularizer=l2(0.0001),
#                                              kernel_constraint=l2(0.0001),
#                                              bias_constraint=l2(0.0001)
                                            ),
                          keras.layers.Dropout(.5),
                          keras.layers.Dense(1, activation='sigmoid', bias_initializer=output_bias)])


    
    model.load_weights(filepath_chkpt)
    predictions = model.predict(scaled_test_samples, batch_size=10, verbose=0)
    rounded_predictions = model.predict_classes(scaled_test_samples, batch_size=10, verbose=0)
    fpr, tpr, thresholds = roc_curve(test_labels, rounded_predictions, pos_label=1)
    auc_score = auc(fpr, tpr)
    list_auc_score.append(auc_score)
    val_epoch = val_epoch + 1
    print('epoch:', val_epoch, 'auc_score:', auc_score)    
    cm = confusion_matrix(test_labels, rounded_predictions)
    
list_epoch = []
for val in range(1,len(list_auc_score) + 1):
    list_epoch.append(val)
    
plt.plot(list_epoch, list_auc_score)

In [ ]:
# TESTING
# creating Modelcheckpoint
checkpoint_path = "./cp.ckpt.testing2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, 
                                                 monitor='val_auc', 
                                                 verbose=1,
                                                 save_best_only=False, 
                                                 save_weights_only=True,
                                                 mode='max',
                                                 save_freq='epoch')


# creating model
epochs = 10
lr = .0001
metrics = [keras.metrics.AUC(name='auc'),
           keras.metrics.FalsePositives(name='fp'),
           keras.metrics.FalseNegatives(name='fn')]

model = keras.Sequential([keras.layers.Dense(16, activation='relu', kernel_regularizer=l2(0.0001), input_shape=(train_samples.shape[-1],)),
                          keras.layers.Dense(16, activation='relu', kernel_regularizer=l2(0.0001),
                                             bias_regularizer=l2(0.0001),
                                             activity_regularizer=l2(0.0001),
#                                              kernel_constraint=l2(0.0001),
#                                              bias_constraint=l2(0.0001)
                                            ),
                          keras.layers.Dropout(.5),
                          keras.layers.Dense(1, activation='sigmoid', bias_initializer=output_bias)])

# Dense(units, activation=None, use_bias=True, kernel_initializer='glorot_uniform',
#       bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None,
#       activity_regularizer=None, kernel_constraint=None, bias_constraint=None
    
model.compile(optimizer=keras.optimizers.Adam(lr=lr), loss=loss, metrics=metrics)


# saving weights for checkpoints
model.save_weights(checkpoint_path.format(epoch=0))


# fitting model
model.fit(scaled_train_samples, train_labels, validation_data=(scaled_val_samples, val_labels), 
          batch_size=20, epochs=epochs, shuffle=True, verbose=2, workers=16, 
          use_multiprocessing=True,
          callbacks=[cp_callback])


# now that all epochs have completed, we will test load and test each weight. 
# parsing through folder with saved weights and selecting unique values with cp-0001 to cp-xxxx to create
# a list of all model weights to parse through for testing results against the test set. 
checkpoint_path = './cp.ckpt.testing2/'
list_checkpoints = []
for file in os.listdir(checkpoint_path):
    val_checkpoint = file[0:12]
    if file[0:3]=='cp-' and val_checkpoint not in list_checkpoints:
        list_checkpoints.append(val_checkpoint)
list_checkpoints.sort()


# parsing through each saved weights through the list, then loading the weights, scoring and plotting results.
filepath_checkpoint_folder = './cp.ckpt.testing2/'
list_auc_score = []
list_epoch = []
val_epoch = 0

for chkpt in list_checkpoints:
    filepath_chkpt = filepath_checkpoint_folder + chkpt

    model = keras.Sequential([keras.layers.Dense(16, activation='relu', kernel_regularizer=l2(0.0001), input_shape=(train_samples.shape[-1],)),
                          keras.layers.Dense(16, activation='relu', kernel_regularizer=l2(0.0001),
                                             bias_regularizer=l2(0.0001),
                                             activity_regularizer=l2(0.0001),
#                                              kernel_constraint=l2(0.0001),
#                                              bias_constraint=l2(0.0001)
                                            ),
                          keras.layers.Dropout(.5),
                          keras.layers.Dense(1, activation='sigmoid', bias_initializer=output_bias)])


    
    model.load_weights(filepath_chkpt)
    predictions = model.predict(scaled_test_samples, batch_size=10, verbose=0)
    rounded_predictions = model.predict_classes(scaled_test_samples, batch_size=10, verbose=0)
    fpr, tpr, thresholds = roc_curve(test_labels, rounded_predictions, pos_label=1)
    auc_score = auc(fpr, tpr)
    list_auc_score.append(auc_score)
    val_epoch = val_epoch + 1
    print('epoch:', val_epoch, 'auc_score:', auc_score)    
    cm = confusion_matrix(test_labels, rounded_predictions)
    
list_epoch = []
for val in range(1,len(list_auc_score) + 1):
    list_epoch.append(val)
    
plt.plot(list_epoch, list_auc_score)

In [ ]:
# TESTING
# creating Modelcheckpoint
checkpoint_path = "./cp.ckpt.testing2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, 
                                                 monitor='val_auc', 
                                                 verbose=1,
                                                 save_best_only=False, 
                                                 save_weights_only=True,
                                                 mode='max',
                                                 save_freq='epoch')


# creating model
epochs = 10
lr = .0001
metrics = [keras.metrics.AUC(name='auc'),
           keras.metrics.FalsePositives(name='fp'),
           keras.metrics.FalseNegatives(name='fn')]

model = keras.Sequential([keras.layers.Dense(16, activation='relu', kernel_regularizer=l2(0.0001), input_shape=(train_samples.shape[-1],)),
                          keras.layers.Dense(16, activation='relu', kernel_regularizer=l2(0.0001),
                                             bias_regularizer=l2(0.0001),
                                             activity_regularizer=l2(0.0001),
#                                              kernel_constraint=l2(0.0001),
#                                              bias_constraint=l2(0.0001)
                                            ),
                          keras.layers.Dropout(.5),
                          keras.layers.Dense(1, activation='sigmoid', bias_initializer=output_bias)])

# Dense(units, activation=None, use_bias=True, kernel_initializer='glorot_uniform',
#       bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None,
#       activity_regularizer=None, kernel_constraint=None, bias_constraint=None
    
model.compile(optimizer=keras.optimizers.Adam(lr=lr), loss=loss, metrics=metrics)


# saving weights for checkpoints
model.save_weights(checkpoint_path.format(epoch=0))


# fitting model
model.fit(scaled_train_samples, train_labels, validation_data=(scaled_val_samples, val_labels), 
          batch_size=20, epochs=epochs, shuffle=True, verbose=2, workers=16, 
          use_multiprocessing=True,
          callbacks=[cp_callback])


# now that all epochs have completed, we will test load and test each weight. 
# parsing through folder with saved weights and selecting unique values with cp-0001 to cp-xxxx to create
# a list of all model weights to parse through for testing results against the test set. 
checkpoint_path = './cp.ckpt.testing2/'
list_checkpoints = []
for file in os.listdir(checkpoint_path):
    val_checkpoint = file[0:12]
    if file[0:3]=='cp-' and val_checkpoint not in list_checkpoints:
        list_checkpoints.append(val_checkpoint)
list_checkpoints.sort()


# parsing through each saved weights through the list, then loading the weights, scoring and plotting results.
filepath_checkpoint_folder = './cp.ckpt.testing2/'
list_auc_score = []
list_epoch = []
val_epoch = 0

for chkpt in list_checkpoints:
    filepath_chkpt = filepath_checkpoint_folder + chkpt

    model = keras.Sequential([keras.layers.Dense(16, activation='relu', kernel_regularizer=l2(0.0001), input_shape=(train_samples.shape[-1],)),
                          keras.layers.Dense(16, activation='relu', kernel_regularizer=l2(0.0001),
                                             bias_regularizer=l2(0.0001),
                                             activity_regularizer=l2(0.0001),
#                                              kernel_constraint=l2(0.0001),
#                                              bias_constraint=l2(0.0001)
                                            ),
                          keras.layers.Dropout(.5),
                          keras.layers.Dense(1, activation='sigmoid', bias_initializer=output_bias)])


    
    model.load_weights(filepath_chkpt)
    predictions = model.predict(scaled_test_samples, batch_size=10, verbose=0)
    rounded_predictions = model.predict_classes(scaled_test_samples, batch_size=10, verbose=0)
    fpr, tpr, thresholds = roc_curve(test_labels, rounded_predictions, pos_label=1)
    auc_score = auc(fpr, tpr)
    list_auc_score.append(auc_score)
    val_epoch = val_epoch + 1
    print('epoch:', val_epoch, 'auc_score:', auc_score)    
    cm = confusion_matrix(test_labels, rounded_predictions)
    
list_epoch = []
for val in range(1,len(list_auc_score) + 1):
    list_epoch.append(val)
    
plt.plot(list_epoch, list_auc_score)